In [ ]:
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torchvision.transforms import Normalize, Resize, ToTensor
from torch.utils.tensorboard import SummaryWriter

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
import os
import time
import datetime
import pandas as pd
from XrayDataset import XrayDataset
from efficientnet_pytorch import EfficientNet



In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [ ]:
torch.cuda.is_available()

In [ ]:
checkpoint = 'checkpoint2_efficientnet'

In [ ]:
log = "efficientnet-b7"

os.path.join(os.getcwd(), 'logs', log)

In [ ]:

tf_writer = SummaryWriter(os.path.join(os.getcwd(), 'logs', log))


In [ ]:

model_name = 'efficientnet-b7'
image_size = EfficientNet.get_image_size(model_name) # 224

In [ ]:
image_size

In [ ]:
def check_rootfolders():
    """Create log and model folder"""
    folders_util = [checkpoint]
    for folder in folders_util:
        if not os.path.exists(folder):
            print('creating folder ' + folder)
            os.mkdir(folder)

check_rootfolders()

In [ ]:
def save_checkpoint(epoch , state):
    filename = '%s/ckpt-%s.pth.tar' % (checkpoint, epoch)
    torch.save(state, filename)

In [ ]:
model = EfficientNet.from_pretrained(model_name)


In [ ]:
model

In [ ]:
model._fc = nn.Sequential(nn.Linear(1280, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 3))


In [ ]:
#model.to(device)
if torch.cuda.is_available():
    model = torch.nn.DataParallel(model).cuda()



In [ ]:

criterion = nn.CrossEntropyLoss()

if torch.cuda.is_available():
    optimizer = optim.Adam(model.parameters(), lr=0.0003)
else:
    optimizer = optim.Adam(model.parameters(), lr=0.0003)



In [ ]:
df = pd.read_csv(os.path.join(os.getcwd(), 'data', 'train.csv'))
df

In [ ]:
df.iloc[0].filename

In [ ]:
len(df.index)

In [ ]:
df.shape[0]

In [ ]:
train_transforms = transforms.Compose([
                                     #transforms.Grayscale(1),
                                      # transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
                                       
    
                                        transforms.RandomHorizontalFlip(p=0.4),
                                        transforms.Resize((224,224)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.5], std=[0.5])
                                       ])
test_transforms = transforms.Compose([
#                                     transforms.Grayscale(1),
                                      transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.5], std=[0.5])
                                      ])

In [ ]:
train_dataset = XrayDataset( 'data', 'train', 'train.csv', train_transforms )

test_dataset = XrayDataset('data', 'test', 'test.csv', test_transforms )

image, label = next(iter(train_dataset))

In [ ]:
label

In [ ]:
batch_size = 64

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size,  shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size,  shuffle=False, num_workers=4)

In [ ]:
image, label = next(iter(train_dataloader))

In [ ]:
def validate(val_loader, model, criterion, epoch):
    model.eval()

    start_val_time = time.time()

    correct = 0
    total = 0
    running_loss = 0.0
    count_loop = 0
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            
            if torch.cuda.is_available():
                input = input.cuda()
                target = target.cuda()
                
            count_loop +=1
            # compute output
            output = model(input)
            loss = criterion(output, target)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

            running_loss += loss.item()

    accurecy = 100 * correct / total
    end_val_time = time.time()

    currentDT = datetime.datetime.now()        
    print(str(currentDT), "=== Validation Loss : ", running_loss, "  Accurecy : ", accurecy, "count loop : ", count_loop, "   Validation Time :  ", (end_val_time-start_val_time) )
    
    if tf_writer is not None:
        tf_writer.add_scalar('accurecy/test', accurecy, epoch)
        tf_writer.add_scalar('loss/test', running_loss, epoch)
    return running_loss

In [ ]:
epochs = 1500
steps = 0
print_every = 1
  
train_losses, test_losses = [], []

In [ ]:
for epoch in range(epochs):
    
    running_loss = 0.0     
    correct = 0
    total = 0 
    
    model.train()

    for inputs, labels in train_dataloader:
        steps += 1
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()
            
        optimizer.zero_grad()
        logps = model(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(logps.data, 1)
        #print(predicted)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #print(correct)
    
    accurecy = 100 * correct / total
    tf_writer.add_scalar('loss/train', running_loss, epoch)
    tf_writer.add_scalar('accurecy/train', accurecy, epoch)
    print( " === epoch : ", epoch, "=== Training Loss : ", running_loss, "  Accurecy : ", accurecy )

    validate(test_dataloader, model, criterion, epoch)
    tf_writer.flush()
    
    
    save_checkpoint(epoch, {
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                
            })

tf_writer.close()
